Load in Libararies

In [1]:
# Importing necessary packages
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import pandas as pd
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

This process will take a minimum of 6_mins_38 seconds depending on how far back you put your start date

In [2]:
# Step 1: Get S&P 500 companies from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]  # The first table contains the tickers
sp500_tickers = sp500_table['Symbol'].tolist()  # Get the list of tickers
sp500_table.set_index('Symbol', inplace=True)  # Set the ticker as index for easier merging later

# Step 2: Get user input for start date
print("Enter the start date in format YYYY-MM-DD (e.g., 2018-01-01):")
start = input()
end = dt.datetime.now().strftime('%Y-%m-%d')  # Current date as the end date

print(f"Fetching historical data for {len(sp500_tickers)} companies in the S&P 500...")

# Step 3: Download historical price data for all tickers from Yahoo Finance
data = yf.download(sp500_tickers, start=start, end=end, interval='1mo', group_by='ticker')

# Step 4: Convert historical data to a Pandas DataFrame
historical_df = data.stack(level=0).reset_index()  # Flatten the multi-level columns
historical_df.rename(columns={'level_1': 'Ticker'}, inplace=True)  # Rename level_1 to Ticker

# Step 5: Fetch additional company data for each ticker
print("Fetching detailed company data...")
financials_list = []
info_list = []

for ticker in sp500_tickers:
    try:
        stock = yf.Ticker(ticker)
        
        # Fetch financials
        financials = stock.financials.T.reset_index()
        financials['Ticker'] = ticker
        financials_list.append(financials)

        # Fetch company info
        info = stock.info
        info['Ticker'] = ticker
        info_list.append(info)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        continue

# Step 6: Combine all the fetched data into DataFrames
financials_df = pd.concat(financials_list, ignore_index=True)
info_df = pd.DataFrame(info_list)

# Step 7: Merge with the original S&P 500 table for additional metadata (like sector/industry)
merged_info_df = sp500_table.merge(info_df, how='left', left_index=True, right_on='Ticker')

# Step 8: Merge the financials_df into the merged_info_df
merged_info_df = merged_info_df.merge(financials_df, how='left', on='Ticker')



Enter the start date in format YYYY-MM-DD (e.g., 2018-01-01):
2024-01-01
Fetching historical data for 503 companies in the S&P 500...


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1mo 2024-01-01 -> 2024-12-07)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Fetching detailed company data...


In [3]:
merged_info_df.head()

,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,address1,city,state,...,Professional Expense And Contract Services Expense,Other Non Interest Expense,Total Other Finance Cost,Rent Expense Supplemental,Rent And Landing Fees,Insurance And Claims,Depletion Income Statement,Net Income From Tax Loss Carryforward,Net Income Extraordinary,Securities Amortization
0,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,3M Center,Saint Paul,MN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,3M Center,Saint Paul,MN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,3M Center,Saint Paul,MN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,3M Center,Saint Paul,MN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,11270 West Park Place,Milwaukee,WI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
merged_info_df["Total Revenue"].head(4)

0    32681000000.0
1    34229000000.0
2    35355000000.0
3    32184000000.0
Name: Total Revenue, dtype: object

In [58]:
merged_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2385 entries, 0 to 2384
Columns: 227 entries, Security to Securities Amortization
dtypes: datetime64[ns](1), float64(99), int64(2), object(125)
memory usage: 4.1+ MB


In [57]:
count_entrities = merged_info_df["Security"].value_counts()
count_entrities 

Intel                             5
Texas Pacific Land Corporation    5
Kenvue                            5
TE Connectivity                   5
Juniper Networks                  5
                                 ..
HP Inc.                           4
GE Vernova                        3
Amentum                           3
Brown–Forman                      1
Berkshire Hathaway                1
Name: Security, Length: 503, dtype: int64

In [5]:
merged_info_df.columns.tolist()

['Security',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Date added',
 'CIK',
 'Founded',
 'address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'irWebsite',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'dividendRate',
 'dividendYield',
 'exDividendDate',
 'payoutRatio',
 'fiveYearAvgDividendYield',
 'beta',
 'trailingPE',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwo

In [12]:
merged_info_df["currentPrice"].head(4)

0    133.11
1    133.11
2    133.11
3    133.11
Name: currentPrice, dtype: float64

In [54]:
merged_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2385 entries, 0 to 2384
Columns: 227 entries, Security to Securities Amortization
dtypes: datetime64[ns](1), float64(99), int64(2), object(125)
memory usage: 4.1+ MB


Below is an attempt to use the Aggregated Dataset to run predictions on Price

In [32]:
merged_info_df_agg = merged_info_df.groupby("Ticker").agg(
    ##Demographic info
    Security=('Security', "last"),
   #GICS Sector=('GICS Sector', "last"),
   #GICS Sub-Industry=('GICS Sub-Industry', "last"),
    Founded=("Founded", "last"),
    Headquarters=("Headquarters Location", "last"),
    City=("city", "last"),
    State = ("state", "last"),
    Zip = ("zip", "last"),
    Country  = ("country", "last"),
    Industry = ("industry", "last"),
    
    ##Logistic info
    
    ##Financial Info
    Median_Gross_Profit = ("Gross Profit", "median"),
    Median_Earning_Quarterly_Growth=('earningsQuarterlyGrowth', "median"),
    Median_EBIDA=('EBITDA', "median"),
    Median_Total_Revenue=('Total Revenue', "median"),
    average_cash_per_share = ('totalCashPerShare', "mean"),
    Price = ("currentPrice", "last")
    
)

C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\jdevr\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=ke

In [33]:
merged_info_df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, A to ZTS
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Security                         503 non-null    object 
 1   Founded                          503 non-null    object 
 2   Headquarters                     503 non-null    object 
 3   City                             501 non-null    object 
 4   State                            481 non-null    object 
 5   Zip                              499 non-null    object 
 6   Country                          501 non-null    object 
 7   Industry                         501 non-null    object 
 8   Median_Gross_Profit              449 non-null    float64
 9   Median_Earning_Quarterly_Growth  441 non-null    float64
 10  Median_EBIDA                     454 non-null    float64
 11  Median_Total_Revenue             501 non-null    float64
 12  average_cash_per_share     

In [34]:
# Count rows with any NaN value
num_rows_with_na = merged_info_df_agg.isna().any(axis=1).sum()

print(f"Number of rows with at least one NA: {num_rows_with_na}")

Number of rows with at least one NA: 125


In [35]:
##Try dropping for right now
# Remove rows with any missing values
merged_info_df_agg_cleaned = merged_info_df_agg.dropna()
merged_info_df_agg_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, A to ZTS
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Security                         378 non-null    object 
 1   Founded                          378 non-null    object 
 2   Headquarters                     378 non-null    object 
 3   City                             378 non-null    object 
 4   State                            378 non-null    object 
 5   Zip                              378 non-null    object 
 6   Country                          378 non-null    object 
 7   Industry                         378 non-null    object 
 8   Median_Gross_Profit              378 non-null    float64
 9   Median_Earning_Quarterly_Growth  378 non-null    float64
 10  Median_EBIDA                     378 non-null    float64
 11  Median_Total_Revenue             378 non-null    float64
 12  average_cash_per_share     

In [36]:
merged_info_df_agg_cleaned.head(10)

,Security,Founded,Headquarters,City,State,Zip,Country,Industry,Median_Gross_Profit,Median_Earning_Quarterly_Growth,Median_EBIDA,Median_Total_Revenue,average_cash_per_share,Price
Ticker,,,,,,,,,,,,,,
A,Agilent Technologies,1999,"Santa Clara, California",Santa Clara,CA,95051,United States,Diagnostics & Research,3.436000e+09,1.541,1.733500e+09,6.576000e+09,6.192,140.58
AAPL,Apple Inc.,1977,"Cupertino, California",Cupertino,CA,95014,United States,Consumer Electronics,1.699650e+11,-0.358,1.281805e+11,3.871600e+11,4.311,242.84
ABBV,AbbVie,2013 (1888),"North Chicago, Illinois",North Chicago,IL,60064-6400,United States,Drug Manufacturers - General,3.632700e+10,-0.122,2.055250e+10,5.525750e+10,4.122,176.19
ABNB,Airbnb,2008,"San Francisco, California",San Francisco,CA,94103,United States,Travel Services,5.868000e+09,-0.687,1.185000e+09,7.195500e+09,18.011,136.92
ABT,Abbott Laboratories,1888,"North Chicago, Illinois",North Chicago,IL,60064-6400,United States,Medical Devices,2.332250e+10,0.146,1.133750e+10,4.159200e+10,4.490,115.60
ADBE,Adobe Inc.,1982,"San Jose, California",San Jose,CA,95110-2704,United States,Software - Infrastructure,1.468050e+10,0.200,6.791000e+09,1.669550e+10,17.072,552.96
ADI,Analog Devices,1965,"Wilmington, Massachusetts",Wilmington,MA,01887,United States,Semiconductors,6.456908e+09,-0.553,4.900270e+09,1.072056e+10,5.127,218.20
ADM,Archer Daniels Midland,1902,"Chicago, Illinois",Chicago,IL,60601,United States,Farm Products,6.750000e+09,-0.476,5.287000e+09,8.959200e+10,1.598,51.42
ADP,Automatic Data Processing,1949,"Roseland, New Jersey",Roseland,NJ,07068,United States,Software - Application,6.915000e+09,0.113,4.820650e+09,1.662265e+10,16.612,304.56


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Step 1: Prepare features and target variable
numeric_columns = [
    "Median_Gross_Profit",
    "Median_Earning_Quarterly_Growth",
    "Median_EBIDA",
    "Median_Total_Revenue",
    "average_cash_per_share",
]
categorical_columns = ["Founded", "Headquarters", "City", "State", "Zip", "Country", "Industry"]


In [38]:
# Normalize numeric columns
scaler = StandardScaler()
merged_info_df_agg_cleaned[numeric_columns] = scaler.fit_transform(merged_info_df_agg_cleaned[numeric_columns])

# One-hot encode categorical columns
merged_info_df_agg_cleaned = pd.get_dummies(merged_info_df_agg_cleaned, columns=categorical_columns, drop_first=False)

merged_info_df_agg_cleaned.head()

C:\Users\jdevr\AppData\Local\Temp\ipykernel_7480\2305009057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_info_df_agg_cleaned[numeric_columns] = scaler.fit_transform(merged_info_df_agg_cleaned[numeric_columns])


,Security,Median_Gross_Profit,Median_Earning_Quarterly_Growth,Median_EBIDA,Median_Total_Revenue,average_cash_per_share,Price,Founded_1806,Founded_1818,Founded_1823,...,Industry_Tools & Accessories,Industry_Travel Services,Industry_Trucking,Industry_Utilities - Diversified,Industry_Utilities - Independent Power Producers,Industry_Utilities - Regulated Electric,Industry_Utilities - Regulated Gas,Industry_Utilities - Regulated Water,Industry_Utilities - Renewable,Industry_Waste Management
Ticker,,,,,,,,,,,,,,,,,,,,,
A,Agilent Technologies,-0.323501,0.845197,-0.336679,-0.368844,-0.100886,140.58,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAPL,Apple Inc.,7.724624,-0.453353,9.556290,5.662970,-0.144327,242.84,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABBV,AbbVie,1.266078,-0.291975,1.135683,0.402701,-0.148692,176.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABNB,Airbnb,-0.205965,-0.678326,-0.379593,-0.359026,0.172072,136.92,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ABT,Abbott Laboratories,0.637588,-0.108715,0.414719,0.186119,-0.140193,115.60,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
merged_info_df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, A to ZTS
Columns: 1280 entries, Security to Industry_Waste Management
dtypes: float64(6), object(1), uint8(1273)
memory usage: 656.7+ KB


In [28]:
merged_info_df_agg.columns.tolist()

['Security',
 'Median_Gross_Profit',
 'Median_Earning_Quarterly_Growth',
 'Median_EBIDA',
 'Median_Total_Revenue',
 'average_cash_per_share',
 'Price',
 'Founded_1784',
 'Founded_1792',
 'Founded_1806',
 'Founded_1810',
 'Founded_1818',
 'Founded_1823',
 'Founded_1825',
 'Founded_1828',
 'Founded_1833',
 'Founded_1837',
 'Founded_1839',
 'Founded_1843',
 'Founded_1845',
 'Founded_1847',
 'Founded_1848',
 'Founded_1849',
 'Founded_1850',
 'Founded_1851',
 'Founded_1852',
 'Founded_1853',
 'Founded_1856',
 'Founded_1858',
 'Founded_1859',
 'Founded_1862',
 'Founded_1866',
 'Founded_1868',
 'Founded_1869',
 'Founded_1870',
 'Founded_1871',
 'Founded_1872',
 'Founded_1875',
 'Founded_1876',
 'Founded_1878',
 'Founded_1879',
 'Founded_1880',
 'Founded_1881/1894 (1980)',
 'Founded_1882',
 'Founded_1883',
 'Founded_1885',
 'Founded_1886',
 'Founded_1888',
 'Founded_1889',
 'Founded_1890',
 'Founded_1891',
 'Founded_1892',
 'Founded_1894',
 'Founded_1896',
 'Founded_1897',
 'Founded_1898',
 'F

In [39]:
# Prepare X and y
X = merged_info_df_agg_cleaned.drop(columns=["Price",  "Security"])  # Exclude non-predictive columns
y = merged_info_df_agg_cleaned["Price"]

X.head()

,Median_Gross_Profit,Median_Earning_Quarterly_Growth,Median_EBIDA,Median_Total_Revenue,average_cash_per_share,Founded_1806,Founded_1818,Founded_1823,Founded_1833,Founded_1837,...,Industry_Tools & Accessories,Industry_Travel Services,Industry_Trucking,Industry_Utilities - Diversified,Industry_Utilities - Independent Power Producers,Industry_Utilities - Regulated Electric,Industry_Utilities - Regulated Gas,Industry_Utilities - Regulated Water,Industry_Utilities - Renewable,Industry_Waste Management
Ticker,,,,,,,,,,,,,,,,,,,,,
A,-0.323501,0.845197,-0.336679,-0.368844,-0.100886,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAPL,7.724624,-0.453353,9.556290,5.662970,-0.144327,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABBV,1.266078,-0.291975,1.135683,0.402701,-0.148692,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABNB,-0.205965,-0.678326,-0.379593,-0.359026,0.172072,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ABT,0.637588,-0.108715,0.414719,0.186119,-0.140193,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.

In [43]:
y.info()

<class 'pandas.core.series.Series'>
Index: 378 entries, A to ZTS
Series name: Price
Non-Null Count  Dtype  
--------------  -----  
378 non-null    float64
dtypes: float64(1)
memory usage: 14.0+ KB


In [41]:

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Step 3: Standardize the full feature set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Linear Regression Model

In [48]:
#Step 4 Define Hyper P for Linear Regression Model
ridge = Ridge()
param_grid = {
    "alpha": [0.1, 1.0, 10.0, 100.0]  # Regularization strength
}

# Step 5: Perform GridSearchCV for optimal parameters
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(X_train_scaled, y_train)
# Step 6: Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
# Results
print("Best Parameters:", grid_search.best_params_)
print("Mean Squared Error on Test Set:", mse)

Best Parameters: {'alpha': 100.0}
Mean Squared Error on Test Set: 129354.38378695521


SVM Regressor

In [50]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Step 4: Define Hyperparameters for SVR
svr = SVR()
param_grid = {
    "kernel": ["linear", "rbf"],         # Types of kernels
    "C": [0.1, 1.0, 10.0, 100.0],       # Regularization parameter
    "epsilon": [0.1, 0.2, 0.5, 0.7]          # Epsilon in the loss function
}

# Step 5: Perform GridSearchCV for optimal parameters
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(X_train_scaled, y_train)

# Step 6: Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)

# Results
print("Best Parameters:", grid_search.best_params_)
print("Mean Squared Error on Test Set:", mse)


Best Parameters: {'C': 1.0, 'epsilon': 0.7, 'kernel': 'linear'}
Mean Squared Error on Test Set: 56813.974725876666


Random Forest Regressor

In [51]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Step 4: Define Hyperparameters for Random Forest
rf = RandomForestRegressor(random_state=42)
param_grid = {
    "n_estimators": [50, 100, 200],     # Number of trees in the forest
    "max_depth": [None, 10, 20, 30],   # Maximum depth of the trees
    "min_samples_split": [2, 5, 10],   # Minimum number of samples required to split a node
    "min_samples_leaf": [1, 2, 4]      # Minimum number of samples required at a leaf node
}

# Step 5: Perform GridSearchCV for optimal parameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Step 6: Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)

# Results
print("Best Parameters:", grid_search.best_params_)
print("Mean Squared Error on Test Set:", mse)


Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error on Test Set: 36793.615110804734


Demo of possible User Inputs

In [3]:
# Function to filter DataFrame based on user selection
def filter_data():
    global filtered_df_eqg  # Declare as global to use outside the function

    # Get the selected filter criteria from the checkboxes
    filter_criteria = {
        'positive_eqg': positive_eqg_var.get(),
        'negative_eqg': negative_eqg_var.get(),
        'positive_ebitda': positive_ebitda_var.get(),
        'negative_ebitda': negative_ebitda_var.get()
    }

    # Start with the full DataFrame
    filtered_df_eqg = merged_info_df

    # Apply filters based on the selected criteria
    if filter_criteria['positive_eqg']:
        filtered_df_eqg = filtered_df_eqg[filtered_df_eqg['earningsQuarterlyGrowth'] > 0]
    if filter_criteria['negative_eqg']:
        filtered_df_eqg = filtered_df_eqg[filtered_df_eqg['earningsQuarterlyGrowth'] < 0]
    if filter_criteria['positive_ebitda']:
        filtered_df_eqg = filtered_df_eqg[filtered_df_eqg['EBITDA'] > 0]
    if filter_criteria['negative_ebitda']:
        filtered_df_eqg = filtered_df_eqg[filtered_df_eqg['EBITDA'] < 0]

    # Clear the Treeview table
    for row in tree.get_children():
        tree.delete(row)

    # Populate the table with the filtered DataFrame
    if filtered_df_eqg is not None and not filtered_df_eqg.empty:
        for _, row in filtered_df_eqg.iterrows():
            tree.insert("", "end", values=row.tolist())
    else:
        tree.insert("", "end", values=("No Data", "N/A"))

# Create the Tkinter GUI
root = tk.Tk()
root.title("Filter Earnings Growth and EBITDA")

# Instruction label
label = tk.Label(root, text="Filter by Earnings or EBITDA:", font=("Arial", 14))
label.pack(pady=10)

# Define checkboxes to hold filter criteria
positive_eqg_var = tk.BooleanVar()
negative_eqg_var = tk.BooleanVar()
positive_ebitda_var = tk.BooleanVar()
negative_ebitda_var = tk.BooleanVar()

# Checkbox for Positive Earnings Growth
positive_eqg_checkbox = tk.Checkbutton(
    root, text="Positive Earnings Growth", variable=positive_eqg_var, onvalue=True, offvalue=False
)
positive_eqg_checkbox.pack(pady=5)

# Checkbox for Negative Earnings Growth
negative_eqg_checkbox = tk.Checkbutton(
    root, text="Negative Earnings Growth", variable=negative_eqg_var, onvalue=True, offvalue=False
)
negative_eqg_checkbox.pack(pady=5)

# Checkbox for Positive EBITDA
positive_ebitda_checkbox = tk.Checkbutton(
    root, text="Positive EBITDA", variable=positive_ebitda_var, onvalue=True, offvalue=False
)
positive_ebitda_checkbox.pack(pady=5)

# Checkbox for Negative EBITDA
negative_ebitda_checkbox = tk.Checkbutton(
    root, text="Negative EBITDA", variable=negative_ebitda_var, onvalue=True, offvalue=False
)
negative_ebitda_checkbox.pack(pady=5)

# Filter Button
filter_button = tk.Button(
    root, text="Apply Filters", command=filter_data, width=20, height=2, bg="lightblue"
)
filter_button.pack(pady=10)

# Frame for Treeview and Scrollbar
frame = tk.Frame(root)
frame.pack(pady=10)

# Treeview to display the DataFrame
columns = list(merged_info_df.columns)
tree = ttk.Treeview(frame, columns=columns, show="headings", height=10, selectmode="browse")

# Set column headings and adjust width
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=150, anchor="center")

# Create a horizontal scrollbar
x_scrollbar = tk.Scrollbar(frame, orient="horizontal", command=tree.xview)
x_scrollbar.pack(side="bottom", fill="x")

# Attach the scrollbar to the treeview
tree.configure(xscrollcommand=x_scrollbar.set)

# Pack the treeview
tree.pack()

# Run the Tkinter event loop
root.mainloop()